In [190]:
import os
import string
import requests
import re
import pandas as pd
import time
from bs4 import BeautifulSoup
from pyvi.ViTokenizer import tokenize

""""Tạo class Word theo cấu trúc dữ liệu của từ điển trong đó có các thuộc tính:
-id_video:- là id của video mô tả ngôn ngữ cử chỉ tương ứng với thuộc tính ý nghĩa-meaning.
pos_tag là gán nhãn từ loại. Các từ loại được gán nhãn bằng thư viện VnCoreNLP sau đó được kiểm tra và chỉnh sửa lại.
Dữ liệu từ điển được lưu trữ trong file DiSL.csv.

Create class Word according to the data structure of the dictionary which has the following properties:
+id_video:- is the id of the video describing the sign language corresponding to the meaning- meaning attribute.
+ pos_tag is the label assignment from the type. The class words are labeled with the VnCoreLNP library and then checked and edited again.
The dictionary data is stored in the file DiSL.csv.

Создание класса Word в соответствии со структурой данных словаря, который имеет следующие свойства:
+id_video:- это идентификатор видео, описывающий язык жестов, соответствующий атрибуту значения- meaning.
+pos_tag — это присвоение метки из типа слова. Слова метки помечаются библиотекой VnCoreLNP,
а затем снова проверяются и редактируются. Данные словаря хранятся в файле DiSL.csv."""
class Word:
    id=''
    id_video=''
    meaning=''
    pos_tag=''
    def __init__(self, id, idvideo,mean, pos_tag):
        self.id=id
        self.id_video=idvideo
        self.pos_tag=pos_tag
        self.meaning=mean
"""
- Đọc dữ liệu từ file csv với pandas. Trả về danh sách my_data là corpus các từ có trong từ điển ngôn ngữ cử chỉ.
key_word_list là danh sách các từ khóa sử dụng để tìm kiếm dữ liệu trên google trong danh sách này không có 2 từ
trùng nhau.Các chữ cái và các chữ số  không được đưa vào tìm kiếm.

- Read data from csv file with pandas. Returns a list of my_data which are the corpus of words in the sign language dictionary. 
key_word_list is a list of keywords used to search for data on google in this list with no duplicate words.
Letters and digits are not included in the search.

- Чтение данных из CSV-файла с помощью pandas. Возвращает список my_data, которые являются корпусом слов в словаре жестового языка.
key_word_list — это список ключевых слов, используемых для поиска данных на google,
в этом списке без повторяющихся слов, буквы и цифры в поиск не включаются.
"""
def read_csv(pathfile):
    my_data=[]
    key_work_list=[]
    df=pd.read_csv(pathfile)
    for i in range(len(df)):
        id=i
        id_video = df.values[i][0]
        mean=df.values[i][1]
        pos_tag=df.values[i][2]
        w=Word(id,id_video,mean,pos_tag)
        my_data.append(w)
        if (pos_tag!='NN')&(pos_tag!='M'):
            word=str(mean).replace('_',' ')
            if (word not in key_work_list):
                key_work_list.append(word)
    return my_data, key_work_list
"""
- Tiền xử lý dữ liệu: xóa thẻ tag html, xóa các ký tự nhiễu, xóa số , phân đoạn từ, loại bỏ stop word.
- Предварительная обработка данных: удаление html-тегов, удаление зашумленных символов,
удаление чисел, сегментация слов, удаление стоп-слов.
- Data preprocessing: remove html tags, remove noise characters, remove numbers, segment words, remove stop words.
Первичная обработка данных"""
def remove_tags(html):
	# parse html content
	soup = BeautifulSoup(html, "html.parser")
	for data in soup(['style', 'script']):
		# Remove tags
		data.decompose()
	# return data by retrieving the tag content
	return ' '.join(soup.stripped_strings)
def clean_text(text):
    text = re.sub('<.*?>', '', text).strip()
    text = re.sub('(\s)+', r'\1', text)
    return text

def normalize_text(text):
    listpunctuation = string.punctuation.replace('_', '')
    for i in listpunctuation:
        text = text.replace(i, ' ')
    return text.lower()


def sentence_segment(text):
    sents = re.split("([.?!])?[\n]+|[.?!] ", text)
    return sents

def word_segment(sent):
    sent = tokenize(sent)
    return sent
def check_vietnamese(document):
    res=False
    if ('có' in document) & ('và' in document) & ('là' in document):
        res= True
    return res
def clean(text):
    new_str = re.sub(r'http\S+', '',text)
    new_str= re.sub("\[.{0,12}\]"," ",new_str)
    context = re.sub("[\s+\.\!\/,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", " ",new_str)
    context = re.sub("[【】╮╯▽╰╭★→「」]+","",context)
    context = re.sub("！，❤。～《》：（）【】म ै थ ि ल ी 」？”“；：、"," ",context)
    context = re.sub("\.*","",context)
    return context
filename = 'stopwords.csv'
data = pd.read_csv(filename, sep="\t", encoding='utf-8')
list_stopwords = data['stopwords']
def remove_stopword(text):
    pre_text = []
    words = text.split()
    for word in words:
        if word not in list_stopwords:
            pre_text.append(word)
        else:
            if word in list_words_search:
                pre_text.append(word)
        text2 = ' '.join(pre_text)
    return text2
"""
- Phương pháp thu thập dữ liệu. 
Để thu thập các ngữ cảnh của các từ ngữ trong từ điển ngôn ngữ ký hiệu,chúng tôi đã tìm kiếm và thu thập các bài
báo từ 3 trang báo lớn và uy tín tại Việt Nam đó là https://dantri.com, https://Vnexpress.net, https://baomoi.com
Sử dụng tìm kiếm google kết hợp với từ khóa tìm kiếm được lấy trong từ điển ngôn ngữ ký hiệu và tên của trang báo, 
sẽ thu được kết quả tìm kiếm trên google là top các bài viết chứa từ khóa tìm kiếm. Để tránh lỗi reques quá nhiều
chúng tôi đã tạo ra thời gian trễ và thay đổi reques giữa các trang báo theo chu kỳ.
Kết quả dữ liệu được tiền xử lý và lưu vào file 'data_train_ok.txt'.

- Data collection methods. To collect the contexts of words in the sign language dictionary,
we searched and collected articles from 3 major and reputable newspapers in Vietnam,
which are https://dantri.com, https://Vnexpress.net, https://baomoi.com. Using google search 
in combination with search keywords taken from the sign language dictionary and the name of the newspaper page, 
the top search results on google will be obtained with the articles containing the search keywords. 
To avoid the error of requests too much we have created a time delay and change requests between pages 
periodically. The resulting data is preprocessed and saved to the file 'data_train_ok.txt'.

- Методы сбора данных. Чтобы собрать контексты слов в словаре жестового языка,
мы провели поиск и собрали статьи из 3 основных и авторитетных газет Вьетнама: https://dantri.com, 
https://Vnexpress.net, https://baomoi.com. Используя поиск Google в сочетании с поисковыми ключевыми словами,
взятыми из словаря жестового языка и названия страницы газеты, лучшие результаты поиска в Google будут получены 
со статьями, содержащими поисковые ключевые слова. Чтобы избежать слишком большого количества запросов,
мы создали временную задержку и периодически меняем запросы между страницами. 
Полученные данные предварительно обрабатываются и сохраняются в файл data_train_ok.txt.
"""
count=0
f_w = open('data_train_ok.txt', 'a')
f_l = open('links_ok.txt', 'a')
def write_data_by_link(link):
    global f_w
    global f_l
    try:
        f_l.write(link + '\n')
        page = requests.get(link,timeout=30)
        content = remove_tags(page.content)
        if (len(content) > 5) & ((check_vietnamese(content))):
            content = clean_text(content)
            sents = sentence_segment(content)
            for sent in sents:
                if (sent != None):
                    sent = word_segment(sent)
                    sent = remove_stopword(normalize_text(sent))
                    sent = normalize_text(sent)
                    if (len(sent.split()) > 1):
                        f_w.write(sent + '\n')
                        # print(clean(sent) + '\n')
    except:
        print("error")
    time.sleep(5)
def write_data(key_word):
    res=[]
    global count
    try:
        from googlesearch import search
    except ImportError:
         print("No module named"+key_word+" found")
    query = ' '.join([key_word,'dantri.com'])
    if count%2==0:
        query = ' '.join([key_word,'Vnexpress.net'])
    if count%3==0:
        query = ' '.join([key_word,'baomoi.net'])
  #query=key_word
    find=False
    while (not find):
        try:
            list_links=search(query,tld="com", num=10, stop=5, pause=2,lang='vi')
            find=True
        except:
            time.sleep(500)
            query = ' '.join([key_word,'baomoi.com'])
            list_links=search(query,tld="com", num=10, stop=5, pause=2,lang='vi')
            if (len(list_links)==5):
                find=True
    for link in list_links:
        if ('facebook'not in link)& ('tim-kiem' not in link)&('https://dantri.com.vn/'!=link) \
            &('https://baomoi.com/'!=link)&('https://vnexpress.net/'!=link):
            res.append(link)
            count=count+1
            write_data_by_link(link)
           #print(str(count)+"\n")
    return res
def get_set(start, step):
    res=[]
    for i in range(start,step+start):
        res.append(list_words_search[i])
    return res

In [191]:
my_data,list_words_search=read_csv('DiSL.csv')

In [ ]:
"""
step=0
start=0
for i in range(len(list_words_search)):
    step=15
    set1=get_set(start,step)
    get_all_link(set1)
    print(str(i)+" done\n")
    time.sleep(500)
    start=start+step
"""

In [192]:
pathdata = './data_train_ok.txt'
pattern = r'[0-9]'
def read_data(path):
    traindata = []
    sents = open(pathdata, 'r').readlines()
    for sent in sents:
        new_sent=re.sub(pattern,'',sent)
        traindata.append(new_sent.split())
    return traindata

In [193]:
train_data=read_data(pathdata)
len(train_data)

301333

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(train_data, vector_size=280, window=10, min_count=2, workers=4, sg=0)
model.wv.save("model/word2vec_skipgram.model")

In [ ]:
from gensim.models.fasttext import FastText
model_fasttext = FastText(vector_size=280, window=10, min_count=2, workers=4, sg=0)
model_fasttext.build_vocab(train_data)
model_fasttext.train(train_data, total_examples=model_fasttext.corpus_count, epochs=model_fasttext.epochs)
model_fasttext.wv.save("model/fasttext_gensim.model")

In [194]:
from gensim.models import KeyedVectors
path_to_model='model/word2vec_skipgram.model'
model = KeyedVectors.load(path_to_model)

In [195]:
def check_word_in_vocab(word):
    res=False
    set_words= model.key_to_index
    if word in set_words:
        res=True
    return res
def join_str(strings):
    return ' '.join(strings)

In [196]:
list_words=[]
new_data=[]
dict_words=dict()
for i in range(len(my_data)):
    if (my_data[i].meaning not in list_words)&(check_word_in_vocab(my_data[i].meaning)):
        list_words.append(my_data[i].meaning)
        new_data.append(my_data[i])
        dict_words[my_data[i].meaning]=my_data[i].pos_tag

In [197]:
def get_words_by_pos_tag(word,pos_tag):
    res=[]
    ls_words=model.most_similar(word,topn=30)
    for w in ls_words:
        if w[0] in list_words:
            if (str(dict_words[str(w[0])])==str(pos_tag))& (model.similarity(str(word),str(w[0]))>0):
                res.append(w[0])
    return res
def check_similarity(w1,w2,e):
    res=False
    if model.similarity(w1,w2)>e:
        res=True
    return res
def check_senten(senten):
    res=True
    ls_senten=senten.split(' ')
    for w in ls_senten:
        if ls_senten.count(w)>1:
            res=False
    return res
    

In [198]:
import random
"""
"""
def general_list_sentents(list_rules,n):
    sentens=[]
    count=0
    while(count<n):
        senten=[]
        arr=[]
        for i in range(len(list_rules)):
            arr.append(get_words_by_pos_tag(list_rules[i][0],list_rules[i][1]))
        for j in range(len(arr)):
            index=random.randint(0,len(arr[j])-1)
            senten.append(str(arr[j][index]))
        senten=join_str(senten)
        #print(senten)
        if (check_senten(senten))&(senten not in sentens):
            sentens.append(senten)
            count=count+1
            #print(str(count)+"++++ "+senten)
    return sentens

In [199]:
list_rules=[['tôi','N'],['yêu','V'],['bạn','N']]
sentens=general_list_sentents(list_rules,20)
for i in range(len(sentens)):
    print(sentens[i])

chị nói_chuyện tình_cảm
bố_mẹ theo_đuổi tình_cảm
thím mơ_ước tình_cảm
tình_cảm nói_chuyện tính_cách
bạn tự_hào tính_cách
tình_cảm nói_chuyện bản_thân
thím mơ_ước lời_nói
anh_chị nói_chuyện tính_cách
chị theo_đuổi bản_thân
thím gần_gũi lời_nói
con_gái theo_đuổi bản_thân
thím yêu_mến bản_thân
anh_chị theo_đuổi tình_cảm
bạn theo_đuổi lời_nói
anh_chị tự_hào bản_thân
bố_mẹ ngưỡng_mộ tình_cảm
trẻ_con gần_gũi tính_cách
bạn mơ_ước lời_nói
bố_mẹ mơ_ước tình_cảm
thím mơ_ước tính_cách


In [84]:
train_data = read_data(pathdata)
list_sentens=train_data[0:5000]
print(list_sentens[0:10])

[['nhiều', 'người', 'thích_thú', 'check', 'in', 'cây', 'thông', 'noel', 'làm', 'từ', 'gần', 'chiếc', 'nón', 'báo', 'dân_trí', 'xã_hội', 'chính_trị', 'học_tập', 'bác', 'môi_trường', 'giao_thông', 'nóng', 'trên', 'mạng', 'sáng_kiến', 'an_toàn', 'giao_thông', 'thế_giới', 'quân_sự', 'hồ_sơ', 'phân_tích', 'thế_giới', 'đó_đây', 'kiều_bào', 'căng_thẳng', 'nga', 'ukraine', 'kinh_doanh', 'tài_chính', 'chứng_khoán', 'doanh_nghiệp', 'khởi_nghiệp', 'tiêu_dùng', 'amaccao', 'doanh_nghiệp', 'tiên_phong', 'thanh_toán', 'thông_minh', 'bất_động_sản', 'dự_án', 'thị_trường', 'nhà_đất', 'nhịp', 'sống', 'đô_thị', 'sống', 'xanh', 'nội_thất', 'thể_thao', 'bóng_đá', 'trong', 'nước', 'bóng_đá', 'châu_âu', 'tennis', 'golf', 'võ_thuật', 'các', 'môn', 'thể_thao', 'khác', 'hậu_trường', 'lịch', 'thi_đấu', 'việc_làm', 'chính_sách', 'làm_giàu', 'chuyện', 'nghề', 'nhân_lực', 'mới', 'nhân_ái', 'danh_sách', 'ủng_hộ', 'danh_sách', 'kết', 'chuyển', 'hoàn_cảnh', 'sức_khỏe', 'ung_thư', 'sống_khỏe', 'dịch_vụ', 'y_tế', 'quốc_t

In [85]:
def find_count_of_word(word,list_sentens):
    count=0
    for senten in list_sentens:
        count=count+senten.count(word)
    return count

In [86]:
def check_words_in_senten(w1,w2,senten):
    return (w1 in senten)&(w2 in senten)

In [87]:
def check_w1_before_w2(w1,w2,c,senten):
    res=False
    if (check_words_in_senten(w1,w2,senten)):
        i1=senten.index(w1)
        i2=senten.index(w2)
        if (i2>i1)&((i2-i1)<=c):
            res=True
    return res

In [88]:
s=find_count_of_word('tháp',list_sentens)
s

2

In [89]:
for senten in list_sentens:
    if (check_w1_before_w2("báo_cáo","khoa_học",3,senten)):
        print(senten)

['gs', 'ước', 'thông_tin', 'thêm', 'trong', 'ngày', 'diễn', 'ra', 'hội_nghị', 'đã', 'có', 'báo_cáo', 'khoa_học', 'và', 'bài', 'giảng', 'của', 'các', 'chuyên_gia', 'hàng_đầu', 'về', 'chuyên_ngành', 'bệnh_lý', 'mạch_máu', 'trong', 'đó', 'có', 'cả', 'việc', 'nhận_diện', 'cấp_cứu', 'căn_bệnh', 'lóc', 'động_mạch', 'chủ', 'nguy_hiểm']


In [90]:
for senten in list_sentens:
    if (check_words_in_senten("sinh","ra",senten)):
        print(senten)

['điều', 'này', 'khiến', 'nhiệt_lượng', 'sinh', 'ra', 'trong', 'quá_trình', 'khai_hỏa', 'cao', 'hơn', 'làm', 'nòng', 'pháo', 'bị', 'tổn_hại', 'nhanh', 'hơn']
['những', 'người', 'hiểu_biết', 'về', 'khoa_học', 'vận_dụng', 'lý_thuyết', 'của', 'nhà', 'vật_lý', 'lừng_danh', 'albert_einstein', 'khẳng_định', 'năng_lượng', 'không', 'tự_nhiên', 'sinh', 'ra', 'và', 'cũng', 'không', 'tự_nhiên', 'mất', 'đi', 'mà', 'nó', 'chỉ', 'chuyển_hóa', 'từ', 'dạng', 'này', 'sang', 'dạng', 'khác']
['điều', 'này', 'dẫn', 'đến', 'sự', 'chuyển_hóa', 'yếm_khí', 'và', 'sinh', 'ra', 'nhiều', 'acid', 'lactic', 'gây', 'đau', 'nhức', 'chân', 'khi', 'vận_động', 'hay', 'còn', 'gọi', 'là', 'đau', 'cách', 'hồi']
['phó_chủ_tịch', 'xã', 'bình_trung', 'vi', 'văn_biên', 'là', 'người', 'xuân_lũng', 'cũng', 'hàng', 'ngày', 'phải', 'đi', 'lai', 'qua', 'dòng', 'kỳ_cùng', 'cho', 'biết', 'từ', 'khi', 'anh', 'sinh', 'ra', 'đến', 'nay', 'đã', 'tuổi', 'người', 'dân', 'vẫn', 'phải', 'dùng', 'bè', 'mảng', 'để', 'đi_lại', 'giao_dịch', 'vớ

In [91]:
def count_w1w2(w1,w2,list_senten):
    count=0
    for senten in list_senten:
        if (check_words_in_senten(w1,w2,senten)):
            count=count+1
    return count
# w1 dung truoc w2 la c_value tu
def count_w1w2_ngram(w1,w2,c_value,list_senten):
    count=0
    for senten in list_senten:
        if (check_w1_before_w2(w1,w2,c_value,senten)):
            count=count+1
    return count

In [28]:
set_words=[]
d=dict()
for senten in list_sentens:
    for w in senten:
        if (w not in set_words)&(w in list_words):
            set_words.append(w)
            d[w]=1
        else:
            if (w in set_words)&(w in list_words):
                d[w]+=1
print(len(set_words))

1312


In [40]:
set_words[0:200]

['thích_thú',
 'in',
 'nón',
 'xã_hội',
 'học_tập',
 'môi_trường',
 'giao_thông',
 'sáng_kiến',
 'phân_tích',
 'doanh_nghiệp',
 'khởi_nghiệp',
 'thông_minh',
 'dự_án',
 'đô_thị',
 'võ_thuật',
 'khác',
 'việc_làm',
 'mới',
 'danh_sách',
 'ủng_hộ',
 'sức_khỏe',
 'ung_thư',
 'dịch_vụ',
 'y_tế',
 'quốc_tế',
 'kiến_thức',
 'điện',
 'đánh_giá',
 'kinh_nghiệm',
 'ô_tô',
 'giáo_dục',
 'gương',
 'sáng',
 'du_học',
 'chuyển_động',
 'tin_tức',
 'khám_phá',
 'tình_yêu',
 'nổi_bật',
 'khoa_học',
 'gửi',
 'quảng_cáo',
 'theo_dõi',
 'thực_hiện',
 'nhà_thờ',
 'ban_đêm',
 'với',
 'chú_ý',
 'xã',
 'huyện',
 'tỉnh',
 'để',
 'trang_trí',
 'm',
 'bằng',
 'tháp',
 'chuông',
 'đặc_biệt',
 'cùng',
 'đỉnh',
 'mỗi',
 'quả',
 'tỉ_mỉ',
 'sắt',
 'họ',
 'ý_tưởng',
 'tháng',
 'truyền_thống',
 'gần_gũi',
 'trước',
 'hoặc',
 'cỏ',
 'chúng_tôi',
 'chia_sẻ',
 'thêm',
 'tuần',
 'tham_quan',
 'hình',
 'thiết_kế',
 'vui_chơi',
 'chị',
 'liên_quan',
 'trung_tâm',
 'phố',
 'khổng_lồ',
 'tre',
 'cơ_quan',
 'hoạt_động',
 'địa_

In [29]:
# xac suat tu i xuat hien cung tu j trong 1 cau va khoang cach c_value tu
w, h = len(set_words)-1, len(set_words)-1
matrix=[[0 for x in range(w)] for y in range(h)] 
def compute_prob(i,j,c_value,list_st):
    global matrix
    if (i==j):
        matrix[i][j]=0
    else:
        if (matrix[i][j]==0):
            f1=count_w1w2_ngram(set_words[i],set_words[j],c_value,list_st)
            f2=count_w1w2_ngram(set_words[j],set_words[i],c_value,list_st)
            #print(set_words[i]+"   "+set_words[j]+"\n")
            s1=d[set_words[i]]
            s2=d[set_words[j]]
            matrix[i][j]=round(1.000*f1/s1,4)
            matrix[j][i]=round(1.000*f2/s2,4)  
   

In [30]:
for i in range(w):
    st = time.time()
    for j in range(i,h):
        compute_prob(i,j,3,list_sentens)
    et = time.time()
    print("done "+str(i)+":"+str(w)+"   "+str(et-st))

done 0:1311   17.550724744796753
done 1:1311   18.55840229988098
done 2:1311   18.944425344467163
done 3:1311   16.43669557571411
done 4:1311   17.205891847610474
done 5:1311   16.87878155708313
done 6:1311   15.832052230834961
done 7:1311   16.297184228897095
done 8:1311   16.433529138565063
done 9:1311   15.898512363433838
done 10:1311   15.962803363800049
done 11:1311   16.161904096603394
done 12:1311   16.51669716835022
done 13:1311   16.371769189834595
done 14:1311   16.39175820350647
done 15:1311   16.632410764694214
done 16:1311   16.430440664291382
done 17:1311   17.850994348526
done 18:1311   16.223984241485596
done 19:1311   16.030916213989258
done 20:1311   16.455141305923462
done 21:1311   16.57658338546753
done 22:1311   16.276106357574463
done 23:1311   16.766150951385498
done 24:1311   16.18139410018921
done 25:1311   16.46996569633484
done 26:1311   16.913872480392456
done 27:1311   16.70290446281433
done 28:1311   15.760485887527466
done 29:1311   16.692365884780884
do

In [179]:
import numpy
matrix_pro=numpy.array(matrix)
pd.DataFrame(matrix_pro).to_csv("matrix_pro1.csv",header=None,index=None)

In [180]:
for j in range(w):
    if (matrix[344][j]>0):
        print(set_words[344]+"   "+ set_words[j]+"   "+ str(matrix[344][j])+"\n")

cấp_cứu   quận   0.0323

cấp_cứu   lấy   0.0323

cấp_cứu   trung_ương   0.0323

cấp_cứu   nhưng   0.0323

cấp_cứu   thành_công   0.0645

cấp_cứu   điều_trị   0.0323

cấp_cứu   kịp_thời   0.0323



In [170]:
check_index=[]
ena=True
def find_next(w,ch):
    global ena
    global check_index
    global matrix
    max_w=0
    res=-1
    for i in range(h):
        if (matrix[w][i]>max_w)&(str(dict_words[set_words[i]])==str(ch))&(not check_index[i]):
            max_w=matrix[w][i]
            res=i
    if (res==-1):
        ena=False
    return res

In [171]:
def general_senten(list_rules,start_word):
    global ena
    global set_words
    global check_index
    index=set_words.index(start_word)
    check_index[index]=True
    i=1
    senten=[]
    senten.append(start_word)
    while (i<len(list_rules))&(ena):
        ind=find_next(index,list_rules[i])
        senten.append(set_words[ind])
        check_index[ind]=True
        index=ind
       #print(str(index) +"  "+ str(ena))
        i+=1
    return ' '.join(senten)
def general_sentens(list_rules,start_word):
    global ena
    res_sentens=[]
    for i in range(w):
        check_index.append(False)
    while (ena):
        senten=general_senten(list_rules,start_word)
        res_sentens.append(senten)
    for i in range(w):
        check_index[i]=False
    ena=True
    return res_sentens


In [172]:
list_rules=['N','V','N','N']
test=general_sentens(list_rules,'thực_phẩm')
for i in range(len(test)-1):
    print(test[i])

thực_phẩm nghe chị hoa
thực_phẩm ăn_uống kỹ_năng bạn


In [173]:
import random
def random_index(pos_tag):
    index=-1
    while (dict_words[set_words[index]]!=pos_tag):
        index=random.randint(0,len(set_words))
    return index
        

In [174]:
i=random_index('N')
print(str(i)+"  "+str(set_words[i]))

1280  hội_trường


In [189]:
count=0
list_rules=['N','A']
while (count<20):
    i=random_index('N') 
    test=general_sentens(list_rules,set_words[i])
    for i in range(len(test)-1):
        print(test[i]) 
    count+=1

sườn buồn_nôn
vàng khổng_lồ
vàng nổi_bật
dự_án dài
dự_án mới
dự_án đầu_tiên
dự_án cao_tốc
quốc_tế mới
tin_tức mới
tin_tức nổi_bật
sườn buồn_nôn
khả_năng thông_minh
khả_năng mới
khả_năng đặc_biệt
khả_năng sâu
khả_năng chính_xác
khả_năng liên_tục
khả_năng nguy_hiểm
khả_năng tự_do
ý_nghĩa đầu_tiên
ý_nghĩa quan_trọng
ý_nghĩa may_mắn
tháng đặc_biệt
tháng đầu_tiên
tháng lâu
tháng muộn
